#### Описание

В этой тетрадке я обучал нейросетки. Очевидно, маловероятно, что классические модели окажутся лучше, чем SotA решения. Единственное препятствие здесь - их большой вес и долгое время работы. С последним ещё всё более-менее, всё таки тексты обычно не очень большие, да и сам текст один, но из-за первой причины модели я так никуда и не залил. Тем не менее, они показали себя лучше и тут я покажу, как этого легко добиться

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import time
from collections import defaultdict
from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import pytorch_lightning as pl
import wandb

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.tuner.tuning import Tuner
from torchmetrics.functional import accuracy
from transformers import AutoTokenizer, AutoModel, AutoConfig
from torch.utils.data import TensorDataset, DataLoader

import random
import os

def seed_everything(seed):
    # Фискирует максимум сидов.
    # Это понадобится, чтобы сравнение оптимизаторов было корректным
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
print("")

No GPU available, using the CPU instead.



## 1. Сбор данных

Как и везде, данные сперва нужно организовать. Тут я уже предварительно собрал датасеты в `ml_training.ipynb`, использую необработанные данные, предполагаю, что встроенные токенизаторы в моделях потом сами разобраться, вроде бы так и надо

In [ ]:
df_train = pd.read_csv('/kaggle/input/imdb-dataframes/train_raw.csv')
df_test = pd.read_csv('/kaggle/input/imdb-dataframes/test_raw.csv')

encoder = LabelEncoder()

df_train["target"] = encoder.fit_transform(df_train["target"])
df_test["target"] = encoder.fit_transform(df_test["target"])

Можно посмотреть на распределение длины предложений, всё-таки большинство нейросетей не смотрит дальше $n$-ого слова, полезно знать, чему это $n$ равно

In [ ]:
df_train["set"] = "train"
df_test["set"] = "test"
df = pd.concat([df_train, df_test])
df["sent_len"] = df.text.apply(lambda x: len(x.split()))

In [18]:
import plotly.express as px

fig = px.histogram(df, x="sent_len", color="set", nbins=200, color_discrete_sequence=["purple", "teal"], )
fig.update_traces(marker_line_width=1, marker_line_color="white")
fig.update_layout(title="Sentence length distribution")

В идеале максимальная длина должна быть 1000 или хотя бы примерно 512, собственно у берта так оно и есть

Далее две функции, одна будет создавать обычно даталоадер, вторая с учётом токенизатора. Понадобятся собственно закодированные токены слов, маски паддинга и таргет, само собой

In [37]:
def create_dataloaders(inputs, masks, labels, batch_size, shuffle=True,
                       use_masks=False):
    input_tensor = torch.tensor(inputs).to(device)
    labels_tensor = torch.tensor(labels).to(device)
    if use_masks:
        mask_tensor = torch.tensor(masks).to(device)
        dataset = TensorDataset(input_tensor, mask_tensor, labels_tensor)
    else:
        dataset = TensorDataset(input_tensor, labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=shuffle)
    return dataloader


def create_clf_dataloaders(df, tokenizer, batch_size,
                           text_columns, target_column,
                           use_masks=True, test=False, test_size=0.2):

    corpus = tokenizer(df[text_columns].tolist(),
              add_special_tokens=True,
              padding='max_length',
              truncation='longest_first',
              max_length=512,
              return_attention_mask=True,
              return_tensors='pt')

    inputs = np.array(corpus['input_ids'])
    masks = np.array(corpus['attention_mask'])
    labels = df[target_column].to_numpy()

    if not test:
        train_inputs, val_inputs, train_labels, val_labels, train_masks, val_masks = \
                    train_test_split(inputs, labels, masks, test_size=test_size, random_state=228)
        train_loader = create_dataloaders(train_inputs, train_masks, 
                                          train_labels, batch_size,
                                          use_masks=use_masks)
        val_loader = create_dataloaders(val_inputs, val_masks, 
                                        val_labels, batch_size,
                                        use_masks=use_masks, shuffle=False)
        return train_loader, val_loader
    else:
        test_loader = create_dataloaders(inputs, masks, 
                                         labels, batch_size,
                                         use_masks=use_masks, shuffle=False)
        return test_loader

И тут же можно объявить абстрактный класс для всех моих будущих моделей, где можно задать шаги обучения. Я везде пользуюсь `pytorch-lightning`, потому что так получается более красивый код

In [38]:
class AbstractDLModel(pl.LightningModule):
    
    def __init__(self, lr):
        super().__init__()
        
        # всё типичное для бинарной классификации
        self.activation = nn.Sigmoid()
        self.loss = nn.BCELoss()
        self.lr = lr
        
    def training_step(self, batch, batch_idx):
        
        # записываем лог-лосс и значение аккураси
        loss, acc = self.step(batch, batch_idx)
        self.log(
            "train_loss",
            loss,
            on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            "train_acc",
            acc,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        
        loss, acc = self.step(batch, batch_idx)
        self.log(
            "val_loss",
            loss,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            "val_acc",
            acc,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss
        
    def configure_optimizers(self):
        # оптимайзер брал дефолтный
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        # шедулер сомнительный, но меня везде устраивает
        scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.5, total_iters=4)
        return {"optimizer": optimizer,
                "lr_scheduler": scheduler}

## 2. LSTM

Самое простое, что можно придумать для работы с текстами, кроме свёрток и полносвязников, это LSTM, конечно же. Сделать её с нуля несложно, но полезность её сомнительна

In [ ]:
pretrained_model = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

Я беру готовый токенизатор для берта, потому что писать его с нуля не очень хочется. Это не особо сложно, но довольно нудно и бесполезно, когда есть уже готовое

Архитектура простая: 
    
1) слой эибеддингов    
2) $n$ полносвязных слоёв с дропаутом (стоило бы ещё добавить батчнорм, но уже поздно)     
3) $m$ слоёв двунаправленной LSTM     
4) сигмоида

In [39]:
class BasicLSTM(AbstractDLModel):
    
    def __init__(self, drop_rate=0.2, lr=1e-4, num_in=512, num_out=1,
                 bidirectional=True, n_layers=1, hidden_dim=300,
                 n_fc=1, len_vocab=28996):
        
        super().__init__(lr=lr)
        self.embed = nn.Embedding(len_vocab, hidden_dim)
        self.linear = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(0.1),
            nn.ReLU()
        )
        self.n_of_fc = n_fc
        self.lstm = nn.LSTM(hidden_dim, hidden_dim,
                            bidirectional=bidirectional,
                            num_layers=n_layers, batch_first=True)
        self.output = nn.Linear(hidden_dim*2*n_layers, 1)
        
    def forward(self, input_ids):
        embeddings = self.embed(input_ids)
        # только сейчас дошло, что это одна и та же сеть
        # или нет?
        for n in range(self.n_of_fc):
            embeddings = self.linear(embeddings)
        # берём все скрытые состояние и конкатенируем
        lstm_h = self.lstm(embeddings)[1][0]
        hidden_flattened = torch.cat([row for row in lstm_h], axis=1)
        return self.activation(self.output(hidden_flattened))
    
    def step(self, batch, batch_idx):
        inputs, labels = batch
        pred = self(inputs).squeeze()
        loss = self.loss(pred, labels.float())
        acc = accuracy(pred, labels, task="binary")
        return loss, acc

Результаты самых удачных экспериментов у меня залиты на wandb, можно сразу глянуть там

In [40]:
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb.login()

wandb: Currently logged in as: lerostre. Use `wandb login --relogin` to force relogin


True

Обучение в лайтнинге делается легко и приятно, учитывая, что там есть автонастройка лёрнинг рейта, моё самое любимое

In [ ]:
seed_everything(123456)

wandb_logger = WandbLogger(project="test_rosatom_bert", log_model='all')
custom_lstm = BasicLSTM(n_fc=0, n_layers=2).to(device)
trainer = pl.Trainer(
    max_epochs=5,
    logger=wandb_logger,
    accelerator="gpu",
#     deterministic=True  почему-то вылетает
)

tuner = Tuner(trainer)
tuner.lr_find(custom_lstm, train_dataloader, val_dataloader,
              update_attr=True)
custom_lstm.lr

In [ ]:
trainer.fit(custom_lstm, train_dataloader, val_dataloader)
wandb.finish()

Конкретно результаты обучения LSTM у меня не сохранились, но выше 0.86 аккураси не поднималась, это совершенно точно, вне зависимости от числа слоёв

## 3. BERT

Дообучить берт стало уже классикой, поэтому почему бы это не сделать

In [4]:
pretrained_model = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = AutoConfig.from_pretrained(pretrained_model)
bert = AutoModel.from_pretrained(pretrained_model, output_hidden_states=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Я на самом деле не знал, что на `huggingface` уже есть готовые шаблоны для классификации, поэтому всегда писал свой, к тому же его можно дополнительно кастомизировать, например, навесить больше полносвязников в конце. Я пробовал это делать, но это всегда приводило к более скорому предобучению, в остальном структура дефолтная:

1) импортированная модель     
2) один полносвязный слой с дропаутом      
3) сигмоида

In [41]:
class ImportedBert(AbstractDLModel):
    
    def __init__(self, inherited_model,
                 drop_rate=0.2, lr=1e-4, num_in=768, num_hid=300, num_out=1, num_fc=1):
        super().__init__(lr)
        
        self.inherited_model = inherited_model
        self.clf_layer = nn.Sequential(
            nn.Linear(num_in, num_hid),
            nn.Dropout(drop_rate),
            nn.ReLU()
        )
        
        for i in range(num_fc):
            linear_block = nn.Sequential(
                nn.Linear(num_hid, num_hid),
                nn.Dropout(drop_rate),
                nn.ReLU()
            )
            self.clf_layer.add_module(f"inter_linear_{i}", linear_block)
                                   
        final_linear = nn.Sequential(
            nn.Linear(num_hid, num_out),
            nn.Sigmoid()
        )
        self.clf_layer.add_module(f"final_linear", final_linear)
        
        if num_fc == 0:
            self.clf_layer = nn.Sigmoid()
        
    def forward(self, input_ids, attention_masks):
        outputs = self.inherited_model(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.clf_layer(class_label_output)
        return outputs
    
    def step(self, batch, batch_idx):
        inputs, masks, labels = batch
        pred = self(inputs, masks).squeeze()
        loss = self.loss(pred, labels.float())
        acc = accuracy(pred, labels, task="binary")
        return loss, acc
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.5, total_iters=4)
        return {"optimizer": optimizer,
                "lr_scheduler": scheduler}

In [ ]:
seed_everything(123456)

wandb_logger = WandbLogger(project="test_rosatom_bert", log_model='all')
custom_bert = ImportedBert(bert, drop_rate=0.2, num_fc=0)
trainer = pl.Trainer(
    max_epochs=5,
    logger=wandb_logger,
    accelerator="gpu",
#     deterministic=True
)

tuner = Tuner(trainer)
tuner.lr_find(custom_bert, train_dataloader, val_dataloader,
              update_attr=True)
custom_bert.lr

In [ ]:
trainer.fit(custom_bert, train_dataloader, val_dataloader)
wandb.finish()

In [14]:
api = wandb.Api()
run = api.run("lerostre/test_rosatom_bert/lxlpoiie")
print("Val accuracy: ", run.summary["val_acc"])

Val accuracy:  0.9240000247955322


Качество получилось уже на 2 процента выше, чем у самой лучшей классической модели, но это явно не предел, можно выжать больше, если обратиться к [статьям](https://paperswithcode.com/sota/sentiment-analysis-on-imdb). Если им верить, можно добиться даже 0.96, если очень постараться

## 4. ROBERTA

Роберта это просто улучшенная модель берта, насколько я могу судить, поэтому логично попробовать обучить её, чтобы побить полученный результат. Тут я столкнулся с тем, что с моим подходом модель не обучалась совсем, поэтому использовал пресет для классификации, хотя в чём разница, я, честно сказать, не вижу

In [14]:
from transformers import RobertaForSequenceClassification

pretrained_model = "cardiffnlp/twitter-roberta-base-emotion"
roberta_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = AutoConfig.from_pretrained(pretrained_model)
roberta = RobertaForSequenceClassification.from_pretrained(pretrained_model, num_labels=1,
                                                           ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Даталоадеры нужно пересказать с новыми токенизаторами

In [8]:
train_dataloader, val_dataloader = create_clf_dataloaders(df_train, roberta_tokenizer, 16,
                                                          "text", "target")

Для нового пресета модель выглядит совсем смешно, но завернуть её всё равно стоит, чтобы трекать метрики как это удобно мне

In [42]:
class ImportedBertClf(ImportedBert):
    
    def __init__(self, inherited_model, lr=1e-4):
        super().__init__(lr)
        
        self.inherited_model = inherited_model
        # у меня уже есть активация, но я об этом забыл, а модель уже обучена
        self.clf_layer = nn.Sigmoid()
        
    def forward(self, input_ids, attention_masks):
        outputs = self.inherited_model(input_ids, attention_masks)["logits"]
        outputs = self.clf_layer(outputs)
        return outputs

На этот раз внизу даже сохранился самый удачный ран

In [ ]:
seed_everything(123456)

wandb_logger = WandbLogger(project="test_rosatom_bert", log_model='all')
custom_roberta = ImportedBertClf(roberta, drop_rate=0.2, num_fc=0)
trainer = pl.Trainer(
    max_epochs=5,
    logger=wandb_logger,
    accelerator="gpu",
#     deterministic=True
)

tuner = Tuner(trainer)
tuner.lr_find(custom_roberta, train_dataloader, val_dataloader,
              update_attr=True)
custom_roberta.lr

In [11]:
trainer.fit(custom_roberta, train_dataloader, val_dataloader)
wandb.finish()

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
train_acc,▁▅██▇
train_loss_epoch,█▄▂▁▂
train_loss_step,▃▂▃▄▆▃▄▃▅▄▃▄▃▄▂▃▂▂▆▁▁▁▂▂▂▂▆▄▁▁▁▁▅▂▆▁▁▁█▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▆▇█▁▅
val_loss,▁▁▁▄█
epoch,4
train_acc,0.9601
train_loss_epoch,0.11155
train_loss_step,0.01824


In [32]:
api = wandb.Api()
run = api.run("lerostre/test_rosatom_bert/2c6nxnts")
print("Val accuracy: ", run.history()["val_acc"].max())

Val accuracy:  0.9376000165939331


Здесь удалось выбить уже 0.938, и на самом деле это самое большое, что удалось получить. Довольно неплохо, но я уверен, что можно сделать больше даже с мощностями кеггла, без использования больших моделей

## 5. XLNet

XLNet это модель, которая выбила самый большой скор на этом датасете, значит, надо её тоже попробовать. На самом деле вынужден признать, что мне не хватает опыта, чтобы сказать, какая из них должна отработать лучше

In [43]:
from transformers import XLNetForSequenceClassification

pretrained_model = "xlnet-base-cased"
xlnet_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = AutoConfig.from_pretrained(pretrained_model)
xlnet = XLNetForSequenceClassification.from_pretrained(pretrained_model, num_labels=1,
                                                       ignore_mismatched_sizes=True)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [15]:
train_dataloader, val_dataloader = create_clf_dataloaders(df_train, xlnet_tokenizer, 16,
                                                          "text", "target")

Обучение полностью идентичное, разве что есть проблемы взрыва градиентов, но это я учёл

In [16]:
seed_everything(123456)

wandb_logger = WandbLogger(project="test_rosatom_bert", log_model='all')
custom_xlnet = ImportedBertClf(xlnet, drop_rate=0.2, num_fc=0)
trainer = pl.Trainer(
    max_epochs=3,
    logger=wandb_logger,
    accelerator="gpu",
    gradient_clip_val=5e6
#     deterministic=True
)

tuner = Tuner(trainer)
tuner.lr_find(custom_xlnet, train_dataloader, val_dataloader,
              update_attr=True)
custom_xlnet.lr

wandb: Currently logged in as: lerostre. Use `wandb login --relogin` to force relogin


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

6.918309709189363e-05

In [ ]:
trainer.fit(custom_xlnet, train_dataloader, val_dataloader)
wandb.finish()

In [33]:
api = wandb.Api()
run = api.run("lerostre/test_rosatom_bert/t1vl4c0t")
print("Val accuracy: ", run.history()["val_acc"].max())

Val accuracy:  0.9345999956130981


Получилось чуть меньше, но скорее всего я накосячил в оптимизации, на это уже просто не остаётся времени. Взял я в итоге XLNet, потому что почему бы и нет, у него качество отстаёт не сильно

## 6. Тест

Остаётся только проверить качество на тестовом датасете

In [47]:
df_train = pd.read_csv('/kaggle/input/imdb-dataframes/train_raw.csv')
df_test = pd.read_csv('/kaggle/input/imdb-dataframes/test_raw.csv')

encoder = LabelEncoder()

df_train["target"] = encoder.fit_transform(df_train["target"])
df_test["target"] = encoder.fit_transform(df_test["target"])

In [48]:
custom_xlnet = ImportedBertClf(xlnet)

Рутина, чтобы импортировать-сохранить-загрузить модель

In [49]:
import wandb
run = wandb.init()
artifact = run.use_artifact('lerostre/test_rosatom_bert/model-t1vl4c0t:v2', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact model-t1vl4c0t:v2, 1342.52MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:1:5.6


In [50]:
checkpoint = torch.load("/kaggle/working/artifacts/model-t1vl4c0t:v2/model.ckpt",
                        map_location=torch.device('cpu'))
custom_xlnet.load_state_dict(checkpoint['state_dict'])
torch.save(custom_xlnet, "xlnet_0.935")

In [51]:
pretrained_model = "xlnet-base-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
test_dataloader = create_clf_dataloaders(df_train, tokenizer, 16, "text", "target", test=True)

Теперь надо оценить её на тестовых данных. Функции точно такие же, как в классическом ноутбуке, всё делалось специально, чтобы было совместимо

In [52]:
%%capture
custom_xlnet = torch.load("/kaggle/working/xlnet_0.935")
custom_xlnet.eval()
custom_xlnet.to(device)

In [53]:
ml_infer = lambda x: ...

def dl_infer(text):

    custom_xlnet.eval()
    corpus = tokenizer([text],
          add_special_tokens=True,
          padding='max_length',
          truncation='longest_first',
          max_length=512,
          return_attention_mask=True,
          return_tensors='pt')

    prob = custom_xlnet(corpus['input_ids'].to(device),
                        corpus['attention_mask'].to(device)).item()
    return prob

def rating_estimate(text, model="logreg"):
    proba = {"logreg": ml_infer, "xlnet": dl_infer}[model](text)
    rating = int(np.round(proba * 10))
    if rating >= 7:
        sentiment = "positive"
    elif rating <= 4:
        sentiment = "negative"
    else:
        sentiment = "mixed"
    return rating, sentiment

In [26]:
xlnet_test, logreg_test = [], []

for i in tqdm(df_test.text.values):
    xlnet_test.append(dl_infer(i))

Инференс долгий. поэтому у меня всё сохранено заранее. Дальше пара импортов из классического ноутбука

In [90]:
def download_datasets(dataset):
    # dataset= ["tokenized", "lemmatized", "raw"]

    X = pd.read_csv(f"/kaggle/input/imdb-dataframes/train_{dataset}.csv")
    X_test = pd.read_csv(f"/kaggle/input/imdb-dataframes/test_{dataset}.csv")

    encoder = LabelEncoder()

    y = encoder.fit_transform(X["target"])
    X = X.drop(["target"], axis=1)

    y_test = encoder.transform(X_test["target"])
    X_test = X_test.drop(["target"], axis=1)
    
    return X, X_test, y, y_test

_, tokenized_test, _, y_test = download_datasets("tokenized")
_, raw_test, _, _ = download_datasets("raw")

In [ ]:
xlnet_test, logreg_test = [], []

for i in tqdm(raw_test.text.values):
    xlnet_test.append(dl_infer(i))

In [106]:
from sklearn.metrics import accuracy_score, roc_auc_score
import pickle

logreg = pickle.load(open("/kaggle/input/imdb-models/logreg_0.905", "rb"))
xlnet_pred = np.load("/kaggle/input/temppppp/xlnet_pred.npy")
logreg_pred = logreg.predict_proba(tokenized_test.text)[:, 1]

print("XLNet")
print(f"Accuracy: {accuracy_score(xlnet_pred > 0.5, y_test)}")
print(f"ROC-AUC: {roc_auc_score(y_test, xlnet_pred)}\n")

print("LogReg")
print(f"Accuracy: {accuracy_score(logreg_pred > 0.5, y_test)}")
print(f"ROC-AUC: {roc_auc_score(y_test, logreg_pred)}\n")

XLNet
Accuracy: 0.93588
ROC-AUC: 0.9805674944

LogReg
Accuracy: 0.9048
ROC-AUC: 0.9653735680000001



In [111]:
import plotly.graph_objects as go
from sklearn.metrics import average_precision_score


def plot_calibration_curve(y_test, preds_list, n_bins=10):
    
    fig = go.Figure()
    for name in preds_list:
        preds = preds_list[name]
        bin_middle_points = []
        bin_real_ratios = []
        n_bins = n_bins
        for i in range(n_bins):
            l = 1.0 / n_bins * i
            r = 1.0 / n_bins * (i + 1)
            bin_middle_points.append((l + r) / 2)
            bin_real_ratios.append(np.mean(y_test[(preds >= l) & (preds < r)])) 
        score = average_precision_score(y_test, preds)
        fig.add_trace(go.Scatter(x=bin_middle_points,
                                 y=bin_real_ratios,
                                 name=f"{name}<br>(PR={score :.5f})", mode='markers+lines'))
        
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0.05, x1=0.95, y0=0.05, y1=0.95
    )
    fig.update_layout(
        title=f"Calibration curves, n_bins={n_bins}",
        xaxis_title="bins",
        yaxis_title="model probability",
        legend_title="Classifier",
        height=500)
    fig.show()

In [112]:
plot_calibration_curve(y_test, {"Optimized LogReg": optimized_proba,
                                "XLNet": xlnet_pred},
                       n_bins=25)

По всем метрикам получилось лучше, чем с логистической регрессией, кроме калибровочной кривой, она, на мой взгляд, выбивается больше, это даже видно по отдельным рейтингам снизу

In [91]:
%%time
print(rating_estimate("This movie is utter dogshit, how could anyone possibly like it???", "xlnet"))
print(rating_estimate("I enjoyed it a lot however", "xlnet"))
print(rating_estimate("It was mediocre for the most part, but the ending was brilliant, dare I say", "xlnet"))

(0, 'negative')
(10, 'positive')
(10, 'positive')
CPU times: user 9.24 s, sys: 935 ms, total: 10.2 s
Wall time: 5.22 s


In [103]:
text = "It’s funny, it’s bright and uplifting, and I think has a lot to say about the modern world – both in terms of feminism and gender equality. Ryan Gosling really goes all-out."

print(dl_infer(text))
print(rating_estimate(text, "xlnet"))

0.9920024871826172
(10, 'positive')
